In [ ]:
# ----------- Pacotes ----------- #
import pandas as pd
import os
import timeit
import numpy as np
import random
import swifter
from pandas_parallel_apply import DataFrameParallel, SeriesParallel


In [ ]:
# ----------- Diretório ----------- #
diretorio = r'C:\Users\Computadores Gamer\OneDrive\Área de Trabalho\apply-if\dados'
diretorio = diretorio.replace('\\','//')

os.chdir(diretorio)
os.getcwd()

In [ ]:
# ----------- Download dos dados ----------- #

#### MORADOR
# largura do txt
larguras = [2,4,1,9,2,1,2,2,1,2,2,4,3,1,1,
            1,1,1,2,1,2,1,1,1,1,1,1,1,1,1,
            1,1,1,1,1,2,1,1,2,1,1,2,1,1,1,
            2,1,2,14,14,10,1,20,20,20,20]

# nome das colunas
colunas = ["UF", "ESTRATO_POF", "TIPO_SITUACAO_REG",
            "COD_UPA", "NUM_DOM", "NUM_UC", "COD_INFORMANTE",
            "V0306", "V0401", "V04021", "V04022", "V04023",
            "V0403", "V0404", "V0405", "V0406", "V0407",
            "V0408", "V0409", "V0410", "V0411", "V0412",
            "V0413", "V0414", "V0415", "V0416",
            "V041711", "V041712", "V041721", "V041722",
            "V041731", "V041732", "V041741", "V041742",
            "V0418", "V0419", "V0420", "V0421", "V0422",
            "V0423", "V0424", "V0425", "V0426", "V0427",
            "V0428", "V0429", "V0430", "ANOS_ESTUDO",
            "PESO", "PESO_FINAL", "RENDA_TOTAL",
            "NIVEL_INSTRUCAO", "RENDA_DISP_PC","RENDA_MONET_PC",
            "RENDA_NAO_MONET_PC","DEDUCAO_PC" ]

# leitura dos dados
MORADOR = pd.read_fwf(
    os.path.join(diretorio, "MORADOR.txt"),
    widths=larguras,
    na_values=[" "],
    names=colunas,
    decimal="."
)

In [ ]:
# ----------- Criação de variável utilizando 'for' e 'if' ----------- #
for i in range(len(MORADOR)):
    if( MORADOR.loc[i,'RENDA_DISP_PC'] + MORADOR.loc[i,'PESO_FINAL']) and\
        (MORADOR.loc[i,'RENDA_MONET_PC']>300) and\
        (MORADOR.loc[i,'RENDA_NAO_MONET_PC']<200) and\
        (MORADOR.loc[i,'RENDA_TOTAL']<1000):
        MORADOR.loc[i,'var_teste'] = 1
    
    elif MORADOR.loc[i,'RENDA_TOTAL']*5 >500 and\
        MORADOR.loc[i,'RENDA_MONET_PC'] > MORADOR.loc[i,'RENDA_TOTAL']/2:
        MORADOR.loc[i,'var_teste'] = 1
    
    else:
        MORADOR.loc[i,'var_teste'] = 0
        
        


In [ ]:
# ----------- Criação de variável utilizando 'apply' ----------- #
def funcao(z):
    if (z['RENDA_DISP_PC'] + z['PESO_FINAL']) and\
        (z['RENDA_MONET_PC']> 300) and\
        (z['RENDA_NAO_MONET_PC']< 200) and\
        z['RENDA_TOTAL']<1000:
        return 1
    
    elif z['RENDA_TOTAL']* 5 > 500 and\
        (z['RENDA_MONET_PC'] > z['RENDA_TOTAL']/2):
        return 1
    
    else:    
        return 0

# aplicação da função
MORADOR['var_teste'] = MORADOR.apply(funcao, axis=1)



In [ ]:
# ----------- Criação de variável utilizando 'np.select' ----------- #
# dados1 = [1,2,3,4,5]
# dados2 = [6,78,34,56,3]
# dados3 = [11,22,44,66,77]

# tabela = pd.DataFrame([dados1, dados2, dados3]).T
# tabela.columns = ['dados1', 'dados2','dados3']
# tabela

# condicoes = [tabela['dados2']>50]
# escolha = [tabela['dados2']]
# resultado = np.select(condicoes, escolha)
# print(resultado)

condicao1 = ((MORADOR['RENDA_DISP_PC'] + MORADOR['PESO_FINAL']>1000) &\
    (MORADOR['RENDA_MONET_PC'] > 300) &\
    (MORADOR['RENDA_NAO_MONET_PC'] < 200) &\
    (MORADOR['RENDA_TOTAL'] < 1000))

condicao2 = ((MORADOR['RENDA_TOTAL'] * 5 > 500) &\
        (MORADOR['RENDA_MONET_PC'] > MORADOR['RENDA_TOTAL'] / 2))

MORADOR['var_teste'] = np.select([condicao1, condicao2], [1, 0])

MORADOR['var_teste'].value_counts()

In [ ]:
# ----------- Criação de variável utilizando 'np.where' ----------- #
# dados1 = [56,67,87]
# dados2 = [1,33,9]
# dados3 = [11,66,55]
# tabela = pd.DataFrame([dados1,dados2,dados3]).T
# tabela.columns = ['dados1','dados2','dados3']

# np.where(tabela['dados2']>20, tabela['dados2'], 'falso')

condicao1 = ((MORADOR['RENDA_DISP_PC'] + MORADOR['PESO_FINAL']>1000) &\
    (MORADOR['RENDA_MONET_PC'] > 300) &\
    (MORADOR['RENDA_NAO_MONET_PC'] < 200) &\
    (MORADOR['RENDA_TOTAL'] < 1000))

condicao2 = ((MORADOR['RENDA_TOTAL'] * 5 > 500) &\
        (MORADOR['RENDA_MONET_PC'] > MORADOR['RENDA_TOTAL'] / 2))


MORADOR['var_teste'] = np.where(condicao1, True , np.where(condicao2, False, 0))


In [ ]:
# ----------- Criação de variável utilizando 'swifter.apply' ----------- #
def funcao(z):
    if (z['RENDA_DISP_PC'] + z['PESO_FINAL'])>1000 and\
        (z['RENDA_MONET_PC']> 300) and\
        (z['RENDA_NAO_MONET_PC']< 200) and\
        z['RENDA_TOTAL']<1000:
        return 1
    
    elif z['RENDA_TOTAL']* 5 > 500 and\
        (z['RENDA_MONET_PC'] > z['RENDA_TOTAL']/2):
        return 1
    
    else:    
        return 0

# aplicação da função
MORADOR['var_teste'] = MORADOR.swifter.apply(funcao, axis=1)

In [ ]:
# ----------- Criação de variável utilizando 'parallel_apply' ----------- #
# def funcao(z):
#     if (z['RENDA_DISP_PC'] + z['PESO_FINAL'])>1000 and\
#         (z['RENDA_MONET_PC']> 300) and\
#         (z['RENDA_NAO_MONET_PC']< 200) and\
#         z['RENDA_TOTAL']<1000:
#         return 1
    
#     elif z['RENDA_TOTAL']* 5 > 500 and\
#         (z['RENDA_MONET_PC'] > z['RENDA_TOTAL']/2):
#         return 1
    
#     else:    
#         return 0

# # aplicação da função
# MORADOR['var_teste'] = MORADOR.parallel_apply(funcao, axis=1)